In [0]:
#Loading raw order data (CSV)
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/FileStore/tables/Amazon_Sale_Report-3.csv")
df.show(5)

+-----+-------------------+----------+--------------------+----------+--------------+------------------+-------+---------------+-------------+----+----------+--------------+---+--------+------+-----------+-----------+----------------+------------+--------------------+-----+------------+-----------+
|index|           Order ID|      Date|              Status|Fulfilment|Sales Channel |ship-service-level|  Style|            SKU|     Category|Size|      ASIN|Courier Status|Qty|currency|Amount|  ship-city| ship-state|ship-postal-code|ship-country|       promotion-ids|  B2B|fulfilled-by|Unnamed: 22|
+-----+-------------------+----------+--------------------+----------+--------------+------------------+-------+---------------+-------------+----+----------+--------------+---+--------+------+-----------+-----------+----------------+------------+--------------------+-----+------------+-----------+
|    0|405-8078784-5731545|2022-04-30|           Cancelled|  Merchant|     Amazon.in|          Stand

In [0]:
#Inspecting schema design
df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Status: string (nullable = true)
 |-- Fulfilment: string (nullable = true)
 |-- Sales Channel : string (nullable = true)
 |-- ship-service-level: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- ASIN: string (nullable = true)
 |-- Courier Status: string (nullable = true)
 |-- Qty: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- ship-city: string (nullable = true)
 |-- ship-state: string (nullable = true)
 |-- ship-postal-code: double (nullable = true)
 |-- ship-country: string (nullable = true)
 |-- promotion-ids: string (nullable = true)
 |-- B2B: boolean (nullable = true)
 |-- fulfilled-by: string (nullable = true)
 |-- Unnamed: 22: boolean (nullable = true)



In [0]:
#Filter out rows with null/invalid data
df = df.dropna(subset=["SKU", "Qty", "Amount", "ship-city"])

In [0]:
#Importing required libraries for next process
from pyspark.sql.functions import col, lit, when, expr, udf
from pyspark.sql.types import StringType, FloatType, IntegerType
import random

# To generate weight, length, width, and height random numbers to use in the process
# random library is used here
spark.udf.register("generate_weight", lambda: round(random.uniform(0.5, 3.0), 2), FloatType())
spark.udf.register("generate_length", lambda: random.randint(20, 40), IntegerType())
spark.udf.register("generate_width", lambda: random.randint(10, 30), IntegerType())
spark.udf.register("generate_height", lambda: random.randint(2, 10), IntegerType())

# Adding those as columns
df = df.withColumn("weight_kg", expr("generate_weight()")) \
       .withColumn("length_cm", expr("generate_length()")) \
       .withColumn("width_cm", expr("generate_width()")) \
       .withColumn("height_cm", expr("generate_height()"))

In [0]:
#Assigning supplier city randomly
supplier_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Hyderabad"]
assign_supplier = udf(lambda: random.choice(supplier_cities), StringType())

df = df.withColumn("supplier_city", assign_supplier())

In [0]:
# Random distance generator
def estimate_distance(supplier, destination):
    if supplier.lower() == destination.lower():
        return 50
    else:
        return random.randint(500, 2000)

estimate_distance_udf = udf(estimate_distance, IntegerType())
df = df.withColumn("distance_km", estimate_distance_udf(col("supplier_city"), col("ship-city")))

In [0]:
df.limit(10).display()

index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22,weight_kg,length_cm,width_cm,height_cm,supplier_city,distance_km
0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,null,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,null,false,Easy Ship,null,2.52,36,23,3,Mumbai,50
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.0,BENGALURU,KARNATAKA,560085.0,IN,"Amazon PLCC Free-Financing Universal Merchant AAT-WNKTBO3K27EJC,Amazon PLCC Free-Financing Universal Merchant AAT-QX3UCCJESKPA2,Amazon PLCC Free-Financing Universal Merchant AAT-5QQ7BIYYQEDN2,Amazon PLCC Free-Financing Universal Merchant AAT-DSJ2QRXXWXVMQ,Amazon PLCC Free-Financing Universal Merchant AAT-CXJHMC2YJUK76,Amazon PLCC Free-Financing Universal Merchant AAT-CC4FAVTYR4X7C,Amazon PLCC Free-Financing Universal Merchant AAT-XXRCW6NZEPZI4,Amazon PLCC Free-Financing Universal Merchant AAT-CXNSLNBROFDW4,Amazon PLCC Free-Financing Universal Merchant AAT-R7GXNZWISTRFA,Amazon PLCC Free-Financing Universal Merchant AAT-WSJLDN3X7KEMO,Amazon PLCC Free-Financing Universal Merchant AAT-VL6FGQVGQVXUS,Amazon PLCC Free-Financing Universal Merchant AAT-EOKPWFWYW7Y6I,Amazon PLCC Free-Financing Universal Merchant AAT-ZYL5UPUNW6T62,Amazon PLCC Free-Financing Universal Merchant AAT-XVPICCHRWDCAI,Amazon PLCC Free-Financing Universal Merchant AAT-ETXQ3XXWMRXBG,Amazon PLCC Free-Financing Universal Merchant AAT-7X3XCTYG64VBE,Amazon PLCC Free-Financing Universal Merchant AAT-7CHGD3WTS3MHM,Amazon PLCC Free-Financing Universal Merchant AAT-26ZDKNME27X42,Amazon PLCC Free-Financing Universal Merchant AAT-4ZF5KN6E4LJK4,Amazon PLCC Free-Financing Universal Merchant AAT-7RCXIKUAX7DDY,Amazon PLCC Free-Financing Universal Merchant AAT-BRSZZ45H6MHAO,Amazon PLCC Free-Financing Universal Merchant AAT-MKLXOOZWQL7GO,Amazon PLCC Free-Financing Universal Merchant AAT-CB7UNXEXGIJTC,Amazon PLCC Free-Financing Universal Merchant #MP-gzasho-1593152694811,Amazon PLCC Free-Financing Universal Merchant AAT-WLBA4GZ52EAH4",false,Easy Ship,null,2.26,26,25,10,Chennai,722
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.0,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,true,null,null,1.2,22,22,8,Delhi,1430
3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,null,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,null,false,Easy Ship,null,1.21,27,22,2,Bangalore,1108
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.0,CHENNAI,TAMIL NADU,600073.0,IN,null,false,null,null,1.98,25,30,8,Delhi,1282
5,404-1490984-4578765,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,SET264,SET264-KR-NP-XL,Set,XL,B08YN7XDSG,Shipped,1,INR,824.0,GHAZIABAD,UTTAR PRADESH,201102.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,false,null,null,0.76,30,30,3,Chennai,973
6,408-5748499-6859555,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,J0095,J0095-SET-L,Set,L,B08CMHNWBN,Shipped,1,INR,653.0,CHANDIGARH,CHANDIGARH,160036.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,false,null,null,0.93,24,14,2,Hyderabad,1247
7,406-7807733-3785945,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3405,JNE3405-KR-S,kurta,S,B081WX4G4Q,Shipped,1,INR,399.0,HYDERABAD,TELANGANA,500032.0,IN,"Amazon PLCC Free-Financing Universal Merchant AAT-WNKTBO3K27EJC,Amazon PLCC Free-Financing Universal Merchant AAT-QX3UCCJESKPA2,Amazon PLCC Free-Financing Universal Merchant AAT-5QQ7BIYYQEDN2,Amazon PLCC Free-Financing Universal Merchant AAT-DSJ2QRXXWXVMQ,Amazon PLCC Free-Financing Universal Merch

In [0]:
#Estimate shipping cost based on weight, distance, and product dimensions
#Formula: cost = weight * distance * rate
rate_per_kg = 0.05
df = df.withColumn("shipping_cost", col("weight_kg") * col("distance_km") * lit(rate_per_kg))

In [0]:
#margin-aware logic
#To offer free shipping above margin threshold, so if the unit price is 

df = df.withColumn("unit_price", (col("Amount") / col("Qty")).cast("float"))
df = df.withColumn("free_shipping", when(col("unit_price") > 500, True).otherwise(False))

In [0]:
#Show final results
df.select("SKU", "ship-city", "supplier_city", "distance_km", "weight_kg", "shipping_cost", "unit_price", "free_shipping").show(10, truncate=False)

+---------------+-----------+-------------+-----------+---------+------------------+----------+-------------+
|SKU            |ship-city  |supplier_city|distance_km|weight_kg|shipping_cost     |unit_price|free_shipping|
+---------------+-----------+-------------+-----------+---------+------------------+----------+-------------+
|SET389-KR-NP-S |MUMBAI     |Chennai      |1248       |2.65     |165.36000976562502|null      |false        |
|JNE3781-KR-XXXL|BENGALURU  |Chennai      |1468       |2.34     |171.75599365234376|406.0     |false        |
|JNE3371-KR-XL  |NAVI MUMBAI|Mumbai       |987        |1.47     |72.54450073242188 |329.0     |false        |
|J0341-DR-L     |PUDUCHERRY |Chennai      |1610       |2.08     |167.43999023437502|null      |false        |
|JNE3671-TU-XXXL|CHENNAI    |Hyderabad    |1856       |2.69     |249.63200683593752|574.0     |true         |
|SET264-KR-NP-XL|GHAZIABAD  |Chennai      |1115       |0.57     |31.77749938964844 |824.0     |true         |
|J0095-SET

In [0]:
df.write.mode("overwrite").parquet("/FileStore/output/shipping_estimation_results")
# For Delta format
# df.write.format("delta").mode("overwrite").save("/FileStore/output/shipping_estimation_results_delta")

In [0]:
df1 = spark.read.option("header", "true").option("inferSchema", "true").parquet("/FileStore/output/shipping_estimation_results")

In [0]:
df1.limit(10).display()

index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22,weight_kg,length_cm,width_cm,height_cm,supplier_city,distance_km,shipping_cost,unit_price,free_shipping
100647,403-3500470-2393109,2022-06-21,Cancelled,Merchant,Amazon.in,Standard,SET350,SET350-KR-NP-XL,Set,XL,B09RKD2Z9R,null,0,INR,971.43,Shillong,MEGHALAYA,793004.0,IN,null,false,Easy Ship,false,2.88,34,27,8,Chennai,1435,206.6400146484375,null,false
100648,406-0054633-6987507,2022-06-21,Shipped,Amazon,Amazon.in,Expedited,JNE3838,JNE3838-KR-XL,kurta,XL,B09TH574ZW,Shipped,1,INR,387.0,KALYAN,MAHARASHTRA,421301.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,false,null,false,1.64,31,13,8,Bangalore,591,48.46199951171875,387.0,false
100649,408-1300504-7482734,2022-06-21,Cancelled,Amazon,Amazon.in,Expedited,SET278,SET278-KR-NP-XS,Set,XS,B0983DSS8R,Unshipped,1,INR,1447.0,JALANDHAR,PUNJAB,144003.0,IN,null,false,null,false,2.89,33,13,7,Chennai,1833,264.86850585937503,1447.0,true
100650,408-1013166-6688339,2022-06-21,Pending,Amazon,Amazon.in,Expedited,JNE3764,JNE3764-KR-M,kurta,M,B09K3TM23C,Unshipped,1,INR,387.0,JAIPUR,RAJASTHAN,302021.0,IN,null,false,null,false,0.85,29,14,5,Chennai,1677,71.27250366210937,387.0,false
100651,404-3840683-4687526,2022-06-21,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,SET293,SET293-KR-NP-L,Set,L,B09K39CKVP,Shipped,1,INR,702.0,BENGALURU,KARNATAKA,560064.0,IN,"Amazon PLCC Free-Financing Universal Merchant AAT-WNKTBO3K27EJC,Amazon PLCC Free-Financing Universal Merchant AAT-QX3UCCJESKPA2,Amazon PLCC Free-Financing Universal Merchant AAT-5QQ7BIYYQEDN2,Amazon PLCC Free-Financing Universal Merchant AAT-DSJ2QRXXWXVMQ,Amazon PLCC Free-Financing Universal Merchant AAT-CXJHMC2YJUK76,Amazon PLCC Free-Financing Universal Merchant AAT-CC4FAVTYR4X7C,Amazon PLCC Free-Financing Universal Merchant AAT-EWAPBB6BDRGM6,Amazon PLCC Free-Financing Universal Merchant AAT-EOKPWFWYW7Y6I,Amazon PLCC Free-Financing Universal Merchant AAT-5SHH3G55USK5C,Amazon PLCC Free-Financing Universal Merchant AAT-5QNBXTEQWGXOW,Amazon PLCC Free-Financing Universal Merchant AAT-BOWEMQBNNNHBK,Amazon PLCC Free-Financing Universal Merchant AAT-SKYXW7T3UZBGO,Amazon PLCC Free-Financing Universal Merchant AAT-ONHQI6RQUNGB6,Amazon PLCC Free-Financing Universal Merchant AAT-7L53RVBBSPI3S,Amazon PLCC Free-Financing Universal Merchant AAT-FXGH5APHPAWRQ,Amazon PLCC Free-Financing Universal Merchant AAT-TNYBHEAO7HOHO,Amazon PLCC Free-Financing Universal Merchant AAT-BRSZZ45H6MHAO,Amazon PLCC Free-Financing Universal Merchant AAT-MKLXOOZWQL7GO,Amazon PLCC Free-Financing Universal Merchant AAT-CB7UNXEXGIJTC,Amazon PLCC Free-Financing Universal Merchant #MP-gzasho-1593152694811,Amazon PLCC Free-Financing Universal Merchant AAT-WLBA4GZ52EAH4,Amazon PLCC Free-Financing Universal Merchant AAT-QMFZ3M7T5NPDS,Amazon PLCC Free-Financing Universal Merchant #MP-pmilan-1601980554292",false,Easy Ship,false,2.19,31,21,5,Chennai,990,108.4050048828125,702.0,true
100653,402-2789243-7189961,2022-06-21,Shipped,Amazon,Amazon.in,Expedited,JNE3373,JNE3373-KR-M,kurta,M,B082W8F8P3,Shipped,1,INR,376.0,BADLAPUR,MAHARASHTRA,421503.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,false,null,false,1.19,21,26,3,Chennai,1597,95.02150268554688,376.0,false
100654,402-3846158-5508359,2022-06-21,Shipped,Amazon,Amazon.in,Expedited,JNE3373,JNE3373-KR-XS,kurta,XS,B082W872VQ,Shipped,1,INR,376.0,PUDUKAD,KERALA,680301.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,false,null,false,2.08,31,18,8,Hyderabad,1736,180.54399414062502,376.0,false
100655,407-3035688-6229153,2022-06-21,Shipped,Amazon,Amazon.in,Expedited,JNE3793,JNE3793-KR-L,kurta,L,B09NQ7HW9W,Shipped,1,INR,349.0,HYDERABAD,TELANGANA,500083.0,IN,Duplicated A12RHGVGRWOT3S 1560498941486,false,null,false,1.88,33,24,5,Delhi,1886,177.28399658203125,349.0,false
100656,406-1511872-9825910,2022-06-21,Shipped - Del